# Logistic Regression


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [2]:
torch.manual_seed(1)    # 같은 결과를 주기 위함

In [3]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [4]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)
print(x_train.shape, y_train.shape)

torch.Size([6, 2]) torch.Size([6, 1])


In [5]:
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

## Hypothesis 계산
$$ H(X) = \frac{1}{1+e^{-W^T X}} $$

In [6]:
hypo = 1 / (1 + torch.exp(-x_train.matmul(W) + b))
print(hypo, hypo.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>) torch.Size([6, 1])


In [7]:
hypo = torch.sigmoid(x_train.matmul(W) + b)
print(hypo, hypo.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>) torch.Size([6, 1])


## Cost function 계산
$$ cost(W) = -\frac{1}{m} \sum y \log\left(H(x)\right) + (1-y) \left( \log(1-H(x) \right) $$

In [8]:
- (y_train[0] * torch.log(hypo[0]) + (1 - y_train[0])*(torch.log(1-hypo[0]))) 

tensor([0.6931], grad_fn=<NegBackward>)

In [9]:
losses = - (y_train * torch.log(hypo) + (1 - y_train)*(torch.log(1-hypo)))
print(losses, losses.shape)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward>) torch.Size([6, 1])


In [10]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


In [11]:
# Computing the Cost Function with F.binary_cross_entropy
F.binary_cross_entropy(hypo,y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward>)

In [12]:
# 모델 초기화
W = torch.zeros((2,1),requires_grad=True)
b = torch.zeros(1,requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W,b],lr = 1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # h(x) 계산
    hypo = torch.sigmoid(x_train.matmul(W)+b)
    # cost 계산
    cost = F.binary_cross_entropy(hypo,y_train)
    # cost로 h(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번 로그마다 출력
    if epoch % 100 == 0:
        print("Epoch: {:4d}/{}, Cost: {:.6f}".format(epoch,nb_epochs,cost.item()))



Epoch:    0/1000, Cost: 0.693147
Epoch:  100/1000, Cost: 0.134722
Epoch:  200/1000, Cost: 0.080643
Epoch:  300/1000, Cost: 0.057900
Epoch:  400/1000, Cost: 0.045300
Epoch:  500/1000, Cost: 0.037261
Epoch:  600/1000, Cost: 0.031672
Epoch:  700/1000, Cost: 0.027556
Epoch:  800/1000, Cost: 0.024394
Epoch:  900/1000, Cost: 0.021888
Epoch: 1000/1000, Cost: 0.019852


## Checking the Accuracy

In [13]:
hypo = torch.sigmoid(x_train.matmul(W) + b)
print(hypo[:5])

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01]], grad_fn=<SliceBackward>)


In [14]:
prediction = hypo >= torch.FloatTensor([0.5])
print(prediction)

tensor([[False],
        [False],
        [False],
        [ True],
        [ True],
        [ True]])


In [15]:
print(prediction)
print(y_train)

tensor([[False],
        [False],
        [False],
        [ True],
        [ True],
        [ True]])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


In [16]:
correct_prediction = prediction.float() == y_train
print(correct_prediction[:5])
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))

tensor([[True],
        [True],
        [True],
        [True],
        [True]])
The model has an accuracy of 100.00% for the training set.


## nn.Module 적용
- self.linear는 W는 8x1, bias 는 1.
    즉 8개의 features
- m = ? , d = 8

In [17]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(8,1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self,x):
        return self.sigmoid(self.linear(x))
model = BinaryClassifier()

In [18]:
# 데이터 불러오기
xy = np.loadtxt('./data-03-diabetes.csv',delimiter=',',dtype=np.float32)
x_data = xy[:,:-1]
y_data = xy[:,[-1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)
print(x_train.shape,y_train.shape)

torch.Size([759, 8]) torch.Size([759, 1])


In [19]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypo = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypo, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypo >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/100 Cost: 0.704829 Accuracy 45.72%
Epoch   10/100 Cost: 0.572392 Accuracy 67.59%
Epoch   20/100 Cost: 0.539564 Accuracy 73.25%
Epoch   30/100 Cost: 0.520041 Accuracy 75.89%
Epoch   40/100 Cost: 0.507561 Accuracy 76.15%
Epoch   50/100 Cost: 0.499125 Accuracy 76.42%
Epoch   60/100 Cost: 0.493177 Accuracy 77.21%
Epoch   70/100 Cost: 0.488846 Accuracy 76.81%
Epoch   80/100 Cost: 0.485612 Accuracy 76.28%
Epoch   90/100 Cost: 0.483146 Accuracy 76.55%
Epoch  100/100 Cost: 0.481234 Accuracy 76.81%
